CASE - 

We are going to build a supervised machine learning system which can predict whether the news is fake or not on the basis of labelled textual data.

WORK FLOW -

Labelled News data

Data PreProcessing

Train Test Split

Logistic Regression Model - We are performing binary classification (Real/Fake)

Trained Logistic Regression Model

Model Evaluation

Feed New Data to model (Test Data)

Prediction

About the Dataset -

train.csv: A full training dataset with the following attributes:

id: unique id for a news article

title: the title of a news article

author: author of the news article

text: the text of the article; could be incomplete

label: a label that marks the article as potentially unreliable

1: Fake News

0: Real News

Importing necessary libraries

In [3]:
import pandas as pd
import numpy as np
import re # regular expression library-usefull for searching words in a text or paragraph
from nltk.corpus import stopwords # natural language toolkit-stopwords-for removing words that doesnt add much value to text or paragraph
from nltk.stem.porter import PorterStemmer # porterstemmer-used to give a root word for particular word
from sklearn.feature_extraction.text import TfidfVectorizer # TfidfVectorizer used to convert text to feature vectors 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SHREE\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [6]:
# Printing the stopwords in english

print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Collection & PreProcessing

In [7]:
# Loading the dataset

data = pd.read_csv(r'C:\Users\SHREE\Downloads\Python CODES\Fake News Prediction Using Logistic Regression Model\train.csv')
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
# Checking the shape of data

data.shape

(20800, 5)

In [9]:
# Checking missing values

data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [11]:
# Replacing null values with empty string

data = data.fillna('') # we can use quote for empty string
data.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [12]:
# Merging the author name & news title

data['content'] = data['author']+' '+data['title']
print(data['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


Separating data & label

In [13]:
x = data.drop(columns = 'label', axis = 1)
y = data['label']

In [14]:
print(x)
print(y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

Stemming Procedure - (PorterStemmer)

The process of reducing a word to its Root Word

EX.

actor, actress, acting (with prefix & suffix) --> act (Root Word), removes prefix & suffix

In [20]:
port_stem = PorterStemmer()

In [21]:
# Explanation of the function


# Creating a function 'stemming' by using 'def' keyword, 'content' represents input we are giving, if we give particular text 
#inside 'stemming' function it will do all process which we have mentioned below

# Then we are calling regular expression library (usefull for searching text/paragraph), then using 'sub' function, it basically
#substitutes certain values, in this case we have mentioned upper arrow it means exclusion, we have mentioned a set 'a-zA-Z'
#for getting all the lower & upper case alphabet & excluding that is not alphabets (ex.removes numbers,punctuations),then we 
#mentioned space ' ',if there is numbers or commas those will be replaced by space thenwe want to remove all the things except
#alphabets from 'content'

# Then we are converting all the alphabets to lower case 'lower()' for better processing

# Then we are splitting it to respective list, all this words are splitted & converted to list

# Then we are stemming these words 'port_stem', we are taking each word & we are removing the stopwords (insignificant words),
#Then we are using for loop to pass in dataframe & remove all the stopwords & choose only the words which are not stopwords,
#those words are reduced to its keyword

# Then we are joining all the words to its stemmed content

# Then we are returning our processed text which are stemmed content

def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [22]:
# Taking the content column (join of author & title), then applying stemming function, this will do all the processing in this
#stemming function & it will returned it to a new column 'content'

data['content'] = data['content'].apply(stemming)

In [23]:
print(data['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


Separating Data & Label

In [24]:
x = data['content'].values
y = data['label'].values

In [25]:
print(x)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [26]:
print(y)

[1 0 1 ... 0 1 1]


In [27]:
y.shape

(20800,)

In [28]:
# Converting textual data to numerical data

vectorizer = TfidfVectorizer() 
# Term frequency inverse document frequency-it counts the no of times a word repeating in document & assings a particular 
#numerical value to that word
#Inverse frequency function-It finds insignificant repeating values & reduces its importance
#Term frequency function-It finds significant repeating values & gives particular value to it
#By this it creates feature vectors-Numbers
vectorizer.fit(x)
x = vectorizer.transform(x)
# Transform will convert all the values to its respective feature

In [29]:
print(x)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

Splitting the Dataset to Train & Test Data

In [30]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify = y, random_state = 2)
# Stratify sagrigates the variable values in equal proportion
# random _state reproduces a particular code

Training Logistic Regression Model -

Sigmoid Function - [y = 1/1+e^-Z] [Z = w.X + b]

We get a 'Sigmoid Curve' by plotting the sigmoid values on y-axis (Prediction Probability) & on x-axis we have to plot input features. It classifies the value based on 'Threshold Value' (middle value).
If it is greater than threshold value then it will classify it as 1 & vice versa.

X - Input feature - Content

Y - Prediction Probability - This value will be between 0-1, this will determine the label will be 0 or 1.

w - Weights - It represents how important a particular feature is.

b - Biases

In [31]:
model = LogisticRegression()

In [32]:
model.fit(x_train, y_train)

LogisticRegression()

Model Evaluation

In [35]:
# Accuracy score on training data

x_train_prediction = model.predict(x_train)
training_data_accuracy = accuracy_score(x_train_prediction, y_train)
print('Accuracy score of training data :', training_data_accuracy)

Accuracy score of training data : 0.9865985576923076


In [36]:
# Accuracy score on testing data

x_test_prediction = model.predict(x_test)
testing_data_accuracy = accuracy_score(x_test_prediction, y_test)
print('Accuracy score of testing data :', testing_data_accuracy)

Accuracy score of testing data : 0.9790865384615385


Conclusion - 

Our model prediction accuracy is very good.

Making a Predictive System

In [40]:
x_news = x_test[1]

prediction = model.predict(x_news)
print(prediction)

if (prediction[0] == 0):
    print('The News is Real')
else:
    print('The News is Fake')

[0]
The News is Real


In [41]:
print(y_test[1])

0
